In [24]:
#!/usr/bin/python
import sys

# input:   each players profit and losses
# output:  each players rake earned
#
# Rule: Player that loses the most gets the rake
# until they are equal with the second biggest loser,
# then they share the rake until they tie the third
# and so on...
def rake_calculator(player_pl):
    rake = -1 * sum(player_pl)
    if (rake < 0):
        raise Exception("Negative Rake.")
    player_pl_with_rake = player_pl[:]
    rake_total = [0]*len(player_pl)
    
    # We distribute rake to the players 
    # that have lost the most until 
    # they tie the next person.
    while(rake > 0):
        
        # Find who deserves rake.
        lowest = min(player_pl_with_rake)
        player_indices_earning_rake = []
        for i in range(len(player_pl)):
            if (player_pl_with_rake[i] == lowest):
                player_indices_earning_rake.append(i)
                
        # Find the second biggest loser, 
        # the one not getting rake.
        second_lowest = 0
        for i in range(len(player_pl)):
            if (player_pl_with_rake[i] > lowest):
                if (player_pl_with_rake[i] < second_lowest):
                    second_lowest = player_pl_with_rake[i]
                    
        # How much rake we are giving out this round.
        # Either enough so we tie the second biggest loser
        # or the rest of the rake.
        giving_cap = (second_lowest-lowest) * len(player_indices_earning_rake)
        amt_to_give = min(giving_cap, rake)
        
        # Give the rake
        amt_per_player = amt_to_give / len(player_indices_earning_rake)
        for i in player_indices_earning_rake:
            rake_total[i] += amt_per_player
            player_pl_with_rake[i] += amt_per_player
        rake -= amt_to_give
        
    return rake_total
      

def payment_list(player_names,player_pl):
    output = ""
    number_of_players = len(player_names)
    if (len(player_pl) != number_of_players):
        raise("player profit lost and name list length mismatch")
    
    # Collect the transactions 
    output = []
    
    # Every iteration of the loop makes a new transaction.
    # The biggest winner pays the biggest loser until one 
    # of them is break even. Then repeat.
    while(max(player_pl) > .001):
        
        # Find the biggest winner and loser
        biggest_winner_amt = max(player_pl)
        biggest_loser_amt = min(player_pl)
        biggest_winner_index = -1
        biggest_loser_index = -1
        for i in range(number_of_players):
            if (player_pl[i] == biggest_winner_amt):
                biggest_winner_index = i
            if (player_pl[i] == biggest_loser_amt):
                biggest_loser_index = i 
        assert(biggest_winner_index != -1)
        assert(biggest_loser_index  != -1)
        
        # Make the payment
        if (biggest_winner_amt > -1*biggest_loser_amt):
            output.append("{0:<18} pays {1:<18} {2:>8.2f}".format(
                                                player_names[biggest_loser_index],
                                                player_names[biggest_winner_index],
                                                -1*biggest_loser_amt))
            player_pl[biggest_winner_index] += biggest_loser_amt
            player_pl[biggest_loser_index]  -= biggest_loser_amt  
        else:
            output.append("{0:<18} pays {1:<18} {2:>8.2f}".format(
                                                player_names[biggest_loser_index],
                                                player_names[biggest_winner_index],
                                                biggest_winner_amt))
            player_pl[biggest_winner_index] -= biggest_winner_amt
            player_pl[biggest_loser_index]  += biggest_winner_amt
    
    # Print out the payments.
    # Sorting organizes by payer.
    output.sort()
    return output
    
# games has format [(player_names,player_pl),...]
def consolidate_pl(games):
    total_pl = {}
    for player_names, player_pl in games:
        for i in range(len(player_names)):
            if player_names[i] in total_pl.keys():
                current = total_pl[player_names[i]]
                total_pl[player_names[i]] = current + player_pl[i]
            else:
                total_pl[player_names[i]] = player_pl[i]
    one_game = [[],[]]
    for player in total_pl.keys():
        one_game[0].append(player)
        one_game[1].append(total_pl[player])
    #print one_game
    return one_game
    
# input format: [[GameName,name1,PL1,name2,PL2,...],[...]]
def main(argv):
    games = []
    for game in argv:
        # Parse the input.
        if (len(game) % 2 == 0):
            print("Player and PL do not match up OR no game name.")
            return
        game_name = game[0]
        number_of_players = (len(game)-1) // 2
        player_names = []
        player_pl = []
        for i in range(0,number_of_players):
            player_names.append(game[2*i+1])
            player_pl.append(float(game[2*i+2]))

        # Give out rake.
        player_rake = rake_calculator(player_pl)
        for i in range(number_of_players):
            player_pl[i] += player_rake[i]
        print("Rake Earned in " + game_name +":")
        for i in range(number_of_players):
            if (player_rake[i] > 0):
                print("{0}: {1:.2f}".format(player_names[i],player_rake[i]))
        print(" ")
        games.append((player_names,player_pl))
    
    one_game = consolidate_pl(games)
    
    output = payment_list(one_game[0],one_game[1])
    for i in output:
        print(i)
    return
        
#if __name__ == "__main__":
#   main(sys.argv[1:])


inpt =[['NLH1',\
        '@Ethan-Leeman',-605.43,\
        '@Csherman42', 0,\
        '@David-Sung-2', -96.84,\
        '@Cole416',0,\
        '@grantwillum',.8,\
        '@Maxwell4727', -300,\
        '@Sajid-Khwaja', 0,\
        '@Santi-Amingad',0,\
        '?Troy-via-Dylan',-557.02,\
        '@rdsilva',1419.44,\
        '@Laurenz-Reyes',0,\
        '@Samuel-Stallop',-218.08,\
        '@noah-savage',268.83,\
        '?shomp',0,\
        '?noah11', 0,\
        '?breakankles',0,\
        '@Darren-Mao',0,\
        '@LilyChao14', 0],
      ['PLO4 HiLo',\
        '@Ethan-Leeman',72.98,\
        '@Csherman42', 0,\
        '@David-Sung-2', 106.57,\
        '@Cole416',0,\
        '@grantwillum',0,\
        '@Maxwell4727', 0,\
        '@Sajid-Khwaja', 0,\
        '@Santi-Amingad',0,\
        '?Troy-via-Dylan',0,\
        '@rdsilva',-197.89,\
        '@Laurenz-Reyes',0,\
        '@Samuel-Stallop',-.8,\
        '@noah-savage',0,\
        '?shomp',0,\
        '?noah11', 0,\
        '?breakankles',0,\
        '@Darren-Mao',0,\
        '@LilyChao14', 0],
      ['NLH Davids Club',\
        '@Ethan-Leeman',-869.72+35.29,\
        '@Csherman42', 0,\
        '@David-Sung-2', 467.90+626.34,\
        '@Cole416',433.38+450.33,\
        '@grantwillum',163.69-288.29,\
        '@Maxwell4727', 637.08-20,\
        '@Sajid-Khwaja', 138.39-358.39,\
        '@Santi-Amingad',500.42-125.14,\
        '?Troy-via-Dylan',-500+705.58,\
        '@rdsilva',662.07-600,\
        '@Laurenz-Reyes',-2006.84-942.29,\
        '@Samuel-Stallop',147.24-91.73,\
        '@noah-savage',453.24,\
        '?shomp',495.57,\
        '?noah11', -29.07,\
        '?breakankles',-60,\
        '@Darren-Mao',-305.29,\
        '@LilyChao14', 0]
      ]
main(inpt)


Rake Earned in NLH1:
@Ethan-Leeman: 68.35
?Troy-via-Dylan: 19.95
 
Rake Earned in PLO4 HiLo:
@rdsilva: 19.14
 
Rake Earned in NLH Davids Club:
@Laurenz-Reyes: 280.24
 
?Troy-via-Dylan    pays ?shomp               331.49
?breakankles       pays @Santi-Amingad        60.00
?noah11            pays ?shomp                 0.71
?noah11            pays @Maxwell4727          18.37
?noah11            pays @Santi-Amingad         9.99
@Darren-Mao        pays @Santi-Amingad       305.29
@Ethan-Leeman      pays @Cole416             883.71
@Ethan-Leeman      pays @noah-savage         414.82
@Laurenz-Reyes     pays @David-Sung-2       1103.97
@Laurenz-Reyes     pays @Maxwell4727         262.16
@Laurenz-Reyes     pays @rdsilva            1302.76
@Sajid-Khwaja      pays @noah-savage         220.00
@Samuel-Stallop    pays ?shomp               163.37
@grantwillum       pays @Maxwell4727          36.55
@grantwillum       pays @noah-savage          87.25
